In [45]:
from glob import glob
import rasterio
import numpy as np
import itertools

In [46]:
imagePath = '/home/shrayank_mistry/Modules/OIF Calculation/Images/'
bands = glob(imagePath+'*B*.jp2')
bands.sort()

In [83]:
band_model = ['02', '03', '04', '05', '06', '07', '11', '12', '8A']
band_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8]

# Calculating Variance of each band

In [49]:
variance_bands = []
for img in bands:
    with rasterio.open(img, 'r') as f:
        arr = np.array(f.read(1))
        N = arr.shape[0] * arr.shape[1]
        a = np.multiply(arr, arr)
        a = np.sum(a)
        a = a / N

        b = np.sum(arr) * np.sum(arr)
        b = b / (N * N)
        variance_bands.append(a - b)

/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in ulong_scalars
  # Remove the CWD from sys.path while we load stuff.


In [50]:
variance_bands

[20371.304029253755,
 30154.93895825586,
 19750.417150034882,
 26103.537382046605,
 27699.754209613748,
 27803.636691173182,
 32494.812155909032,
 25867.83243001975,
 26011.627844344097]

# Calculate Covariance matrix

In [51]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

In [52]:
bands_combi_2 = findsubsets(band_numbers, 2)

In [64]:
covariance_bands = []
for b in bands_combi_2:
    b1, b2 = None, None
    with rasterio.open(bands[b[0]], 'r') as f:
        b1 = np.array(f.read(1))
    
    with rasterio.open(bands[b[1]], 'r') as f:
        b2 = np.array(f.read(1))
    
    N = b1.shape[0] * b1.shape[1]

    a = np.multiply(b1, b2)
    a = np.sum(a)
    a = a / N

    b = np.sum(b1) / N
    c = np.sum(b2) / N

    covariance_bands.append(a - (b * c))


# Calculate Correlation Matrix

In [67]:
import math

In [77]:
correlation_bands = np.zeros((len(bands_combi_2), len(bands_combi_2)))
i = 0
for b in bands_combi_2:
    b1, b2 = b
    correlation_bands[b1][b2] = covariance_bands[i] / math.sqrt(variance_bands[b1] * variance_bands[b2])
    i = i + 1

In [78]:
std_bands = np.sqrt(variance_bands)

In [85]:
std_bands

array([142.72807723, 173.65177499, 140.53617737, 161.56589177,
       166.43243136, 166.74422536, 180.26317471, 160.83479857,
       161.28120735])

In [81]:
bands_combi_3 = findsubsets(band_numbers, 3)

In [87]:
oif_history = []
for b in bands_combi_3:
    b1, b2, b3 = b
    x, y, z = correlation_bands[b1][b2], correlation_bands[b1][b3], correlation_bands[b2][b3], 
    oif = (std_bands[b1] + std_bands[b2] + std_bands[b3]) / (np.abs(x) + np.abs(y) + np.abs(z))
    oif_history.append(f'OIF [Band Models = ({band_model[b1]}, {band_model[b2]}, {band_model[b3]})] = {oif}')
    print(f'OIF [Band Models = ({band_model[b1]}, {band_model[b2]}, {band_model[b3]})] = {oif}')

OIF [Band Models = (02, 03, 04)] = 4.563241049321775
OIF [Band Models = (02, 03, 05)] = 4.235337150279519
OIF [Band Models = (02, 03, 06)] = 3.491869138095041
OIF [Band Models = (02, 03, 07)] = 3.221795222973473
OIF [Band Models = (02, 03, 11)] = 3.001004266764592
OIF [Band Models = (02, 03, 12)] = 3.3603166142599155
OIF [Band Models = (02, 03, 8A)] = 2.8368519317613328
OIF [Band Models = (02, 04, 05)] = 2.9356749095730668
OIF [Band Models = (02, 04, 06)] = 2.440594923602215
OIF [Band Models = (02, 04, 07)] = 2.2578316967853373
OIF [Band Models = (02, 04, 11)] = 2.1138169546311314
OIF [Band Models = (02, 04, 12)] = 2.348554569256499
OIF [Band Models = (02, 04, 8A)] = 1.9929908460802226
OIF [Band Models = (02, 05, 06)] = 2.2922005182944187
OIF [Band Models = (02, 05, 07)] = 2.122660298408405
OIF [Band Models = (02, 05, 11)] = 1.987219349554122
OIF [Band Models = (02, 05, 12)] = 2.2075820476962056
OIF [Band Models = (02, 05, 8A)] = 1.8770855712343382
OIF [Band Models = (02, 06, 07)] = 1.

In [88]:
print(oif_history)

['OIF [Band Models = (02, 03, 04)] = 4.563241049321775', 'OIF [Band Models = (02, 03, 05)] = 4.235337150279519', 'OIF [Band Models = (02, 03, 06)] = 3.491869138095041', 'OIF [Band Models = (02, 03, 07)] = 3.221795222973473', 'OIF [Band Models = (02, 03, 11)] = 3.001004266764592', 'OIF [Band Models = (02, 03, 12)] = 3.3603166142599155', 'OIF [Band Models = (02, 03, 8A)] = 2.8368519317613328', 'OIF [Band Models = (02, 04, 05)] = 2.9356749095730668', 'OIF [Band Models = (02, 04, 06)] = 2.440594923602215', 'OIF [Band Models = (02, 04, 07)] = 2.2578316967853373', 'OIF [Band Models = (02, 04, 11)] = 2.1138169546311314', 'OIF [Band Models = (02, 04, 12)] = 2.348554569256499', 'OIF [Band Models = (02, 04, 8A)] = 1.9929908460802226', 'OIF [Band Models = (02, 05, 06)] = 2.2922005182944187', 'OIF [Band Models = (02, 05, 07)] = 2.122660298408405', 'OIF [Band Models = (02, 05, 11)] = 1.987219349554122', 'OIF [Band Models = (02, 05, 12)] = 2.2075820476962056', 'OIF [Band Models = (02, 05, 8A)] = 1.8